In [ ]:
!pip install nltk

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()

text = ""

sentiment_scores = sid.polarity_scores(text)

if sentiment_scores['compound'] >= 0.05:
    sentiment = 'Positive'
elif sentiment_scores['compound'] <= -0.05:
    sentiment = 'Negative'
else:
    sentiment = 'Neutral'

print(f"Text: {text}")
print(f"Sentiment: {sentiment}")
print(f"Sentiment Scores: {sentiment_scores}")


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Text: 
Sentiment: Neutral
Sentiment Scores: {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}


In [ ]:
!pip install beautifulsoup4 matplotlib nltk

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from urllib.request import urlopen
from urllib.request import Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer
tickers = ['AAPL', 'TSLA', 'AMZN', 'PLTR', 'NVDA', 'WBA']

In [ ]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

finviz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'

for ticker in tickers:
    url = finviz_url + ticker
    req = Request(url=url, headers={'User-Agent': user_agent})

    try:
        resp = urlopen(req)
        html = BeautifulSoup(resp, features="lxml")
        news_table = html.find(id='news-table')
        news_tables[ticker] = news_table

        df = news_tables[ticker]
        df_tr = df.findAll('tr')

        print ('\n')
        print ('Recent News Headlines for {}: '.format(ticker))

        for i, table_row in enumerate(df_tr):
            a_text = table_row.a.text
            td_text = table_row.td.text
            td_text = td_text.strip()
            print(a_text,'(',td_text,')')
            if i == n-1:
                break
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")




Recent News Headlines for AAPL: 
iPhone 15 Pro review 3 months later: price, performance ( Today 12:15PM )
Error fetching data for AAPL: name 'n' is not defined


Recent News Headlines for TSLA: 
Sustainable Finance Newsletter - Precedent for Musk's F-bomb ( Today 12:00PM )
Error fetching data for TSLA: name 'n' is not defined


Recent News Headlines for AMZN: 
Amazon Targets Shein With Fee Cuts for Cheap Apparel Sellers ( Today 12:05PM )
Error fetching data for AMZN: name 'n' is not defined


Recent News Headlines for PLTR: 
Palantir Stock Keeps Dropping. It Just Failed a Key Test. ( Today 10:11AM )
Error fetching data for PLTR: name 'n' is not defined


Recent News Headlines for NVDA: 
Nvidia Says It Will Make Chips That Comply With US Export Rules ( Today 12:01PM )
Error fetching data for NVDA: name 'n' is not defined


Recent News Headlines for WBA: 
Walgreens Boots Alliance (WBA) Flat As Market Sinks: What You Should Know ( Dec-04-23 05:45PM )
Error fetching data for WBA: name '

In [ ]:
parsed_news = []

for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
        a_tag = x.a
        if a_tag:
            text = a_tag.get_text()
            date_scrape = x.td.text.split()

            if len(date_scrape) == 1:
                time = date_scrape[0]
            else:
                date = date_scrape[0]
                time = date_scrape[1]

            ticker = file_name.split('_')[0]

            parsed_news.append([ticker, date, time, text])


In [ ]:
analyzer = SentimentIntensityAnalyzer()

columns = ['Ticker', 'Date', 'Time', 'Headline']
news = pd.DataFrame(parsed_news, columns=columns)
scores = news['Headline'].apply(analyzer.polarity_scores).tolist()

df_scores = pd.DataFrame(scores)
news = news.join(df_scores, rsuffix='_right')

In [ ]:
import pandas as pd
from datetime import datetime

today_date = datetime.today().date()

news['Date'] = news['Date'].replace('Today', today_date)

news['Date'] = pd.to_datetime(news['Date']).dt.date

unique_ticker = news['Ticker'].unique().tolist()
news_dict = {name: news.loc[news['Ticker'] == name] for name in unique_ticker}

values = []
for ticker in tickers:
    dataframe = news_dict[ticker]
    dataframe = dataframe.set_index('Ticker')
    dataframe = dataframe.drop(columns=['Headline'])
    print('\n')
    print(dataframe.head())

    mean = round(dataframe['compound'].mean(), 2)
    values.append(mean)

df = pd.DataFrame(list(zip(tickers, values)), columns=['Ticker', 'Mean Sentiment'])
df = df.set_index('Ticker')
df = df.sort_values('Mean Sentiment', ascending=False)
print('\n')
df



              Date     Time    neg    neu    pos  compound
Ticker                                                    
AAPL    2023-12-06  12:15PM  0.000  1.000  0.000    0.0000
AAPL    2023-12-06  11:26AM  0.273  0.727  0.000   -0.4588
AAPL    2023-12-06  11:21AM  0.000  1.000  0.000    0.0000
AAPL    2023-12-06  11:21AM  0.000  0.680  0.320    0.5106
AAPL    2023-12-06  10:51AM  0.000  0.781  0.219    0.6369


              Date     Time  neg    neu    pos  compound
Ticker                                                  
TSLA    2023-12-06  12:00PM  0.0  1.000  0.000    0.0000
TSLA    2023-12-06  11:26AM  0.0  1.000  0.000    0.0000
TSLA    2023-12-06  10:55AM  0.0  1.000  0.000    0.0000
TSLA    2023-12-06  10:27AM  0.0  1.000  0.000    0.0000
TSLA    2023-12-06  09:58AM  0.0  0.787  0.213    0.4019


              Date     Time    neg    neu    pos  compound
Ticker                                                    
AMZN    2023-12-06  12:05PM  0.185  0.672  0.143   -0.1280
AMZN 

,Mean Sentiment
Ticker,
PLTR,0.28
AMZN,0.24
NVDA,0.20
AAPL,0.15
TSLA,0.09
WBA,0.03
